# PHYS 325 Scientific Computing -- Fall 2018

 ## 1.4 Numerical basics


### 1.4.1 Algorithm

=> a set of steps to be followed in calculations or other problem-solving operations, especially by a computer

**Example:** scalar product $s$ of two vectors $\mathbf{x}, \mathbf{y}\in\mathbb{R}^N$

- Mathematical expression: $$s=\mathbf{x}\cdot\mathbf{y}=\sum_{i=1}^Nx_iy_i$$
- Algorithm in words:
    - start at $s=0$
    - for each element $i$ of the vectors $\mathbf{x}$ and $\mathbf{y}$:
    - calculate $x_i$ times $y_i$
    - add this product to $s$
- Python code
  ```python
     s = 0
     for i in range(len(x)):
         s += x[i]*y[i]
  ```

### 1.4.2 Computational complexity

When analyzing algorithms we are interested in
- how much CPU time is required?
- how much memory is required?
- how big is the bandwidth (transfer of data)?

> How do these numbers scale with the input size?

**Example**: addition as taught in primary school
$$
\begin{array}{lr}
 & 215\\
+& 837\\
\hline
 & 1052
\end{array}
$$

- size of input: $n$ digits per input number
- however there are also $n$ carry digits
- so we have to perform $2n$ additions => linear scaling as function of input length
> time required: $T(n) = 2n$

**Example**: multiplication as taught in primary school
$$
\begin{array}{lr}
 & 151\\
\times & 175\\
\hline
 & 755\\
+ & 1057\hphantom{0}\\
 & 151\hphantom{00}\\
\hline
& 26425
\end{array}
$$

- size of input: $n$ digits per input number
- number of multiplications: $n\cdot n = n^2$
- number of additions: also proportional to $n^2$ => quadratic scaling as function of input length
> time required: $T(n) = n^2 + an^2 = cn^2$

Quadratic always beats linear for sufficiently large $n$:

![quadratic vs linear scaling](images/quadratic_linear.png)

- we are usually interested in the large $n$ behavior (asymptotic behavior)
- but for small $n$ beware of the constant prefactor!

$\mathcal{O}(N)$ notation for time complexity
- applies to algorithm, not specific machine or implementation
- how long a job runs always depends on the computer

Examples:

$$
\begin{array}{rcl}
n^2 &=& \mathcal{O}(n^2)\\
n^2 + 2n &=& \mathcal{O}(n^2)\\
n^2 + \log(n) &=& \mathcal{O}(n^2)\\
5n^2 &=& \mathcal{O}(n^2)
\end{array}
$$

assuming 10 GFlop (about 2.5 GHz processor) => one operation takes about 0.1 ns

complexity | $N=10$ | $10^2$  | $10^3$   | $10^4$  | $10^5$ | $10^6$    
-----------|--------|---------|----------|---------|--------|-----------
1          | 0.1 ns | 0.1 ns  | 0.1 ns   | 0.1 ns  | 0.1 ns | 0.1 ns     
log $N$    | 0.3 ns | 0.7 ns  | 1.0 ns   | 1.3 ns  | 1.7 ns | 2.0 ns    
$N$        | 1 ns   | 10 ns   | 100 ns   | 1 μs    | 10 μs  | 0.1 ms    
$N$log$N$  | 3.3 ns | 66.4 ns | 1 μs     | 13 μs   | 0.17 ms| 2 ms      
$N^2$      | 10 ns  | 1 μs    | 0.1 ms   | 10 ms   | 1 s    | 1.7 min   
$N^3$      | 0.1 μs | 0.1 ms  | 0.1 s    | 1.7 min | >1 day | >3 years  
$2^N$      | 0.1 μs | $10^{12}$ years | $10^{283}$ years |        
$N!$       | 0.4 ms | $10^{140}$ years |
 
=> reducing the computational complexity when possible is crucial for efficiency!

**Example**: Is matrix multiplication an $N^3$ process?
<div style="text-align: right"> based on Ch. 2.11 from "Numerical Recipes in C"</div>

How many individual multiplications does it take to multiply two $2\times 2$ matrices?

$$
\begin{pmatrix}
a_{11} & a_{12} \\
a_{21} & a_{22}
\end{pmatrix}\cdot
\begin{pmatrix}
b_{11} & b_{12} \\
b_{21} & b_{22}
\end{pmatrix}
=
\begin{pmatrix}
c_{11} & c_{12} \\
c_{21} & c_{22}
\end{pmatrix}
$$

The straightforward answer is, of course, eight:

$$
\begin{array}{rcl}
c_{11} &=& a_{11}\cdot b_{11} + a_{12}\cdot b_{21}\\
c_{12} &=& a_{11}\cdot b_{12} + a_{12}\cdot b_{22}\\
c_{21} &=& a_{21}\cdot b_{11} + a_{22}\cdot b_{21}\\
c_{22} &=& a_{21}\cdot b_{12} + a_{22}\cdot b_{22}
\end{array}
$$

Can one write formulas for the $c$'s that involve only *seven* multiplications?

> Yes, as discovered by Volker Strassen [Strassen, V. 1969, Numerische Mathematik, vol. 13, pp. 354-356](https://link.springer.com/article/10.1007%2FBF02165411)

$$
\begin{array}{rcl}
Q_1 &=& (a_{11} + a_{22})\cdot (b_{11} + b_{22})\\
Q_2 &=& (a_{21} + a_{22})\cdot b_{11}\\
Q_3 &=& a_{11}\cdot (b_{12} - b_{22})\\
Q_4 &=& a_{22}\cdot (-b_{11} + b_{21})\\
Q_5 &=& (a_{11} + a_{12})\cdot b_{22}\\
Q_6 &=& (-a_{11} + a_{21})\cdot (b_{11} + b_{12})\\
Q_7 &=& (a_{12} - a_{22})\cdot (b_{21} + b_{22})
\end{array}
$$
in terms of which
$$
\begin{array}{rcl}
c_{11} &=& Q_1 + Q_4 - Q_5 + Q_7\\
c_{12} &=& Q_2 + Q_4\\
c_{21} &=& Q_3 + Q_5\\
c_{22} &=& Q_1 + Q_3 - Q_2 + Q_6
\end{array}
$$

What's the use of this?

=> there are many more additions and subtractions instead!

- these equations are valid also if the $a$'s and $b$'s are matrices
- large $2^m\times 2^m$ matrices can be multiplied by partitioning them into quarters, sixteenth, etc.
- the savings "7/8" applies at **each** level of partitioning

=> reduces the complexity to $\mathcal{O}(N^{\log_2 7})\approx\mathcal{O}(N^{2.8})$ instead of $\mathcal{O}(N^3)$

What is the fastest algorithm for matrix multiplication?

> => still unsolved!

<img src="images/Bound_on_matrix_multiplication_omega_over_time.jpg" alt="matrix multiplication algorithms" style="width: 500px;"/>
<br>
<div style="text-align: center"> Lowest $\omega$ such that matrix multiplication is known to be in $\mathcal{O}(n^\omega)$, plotted against time<br>Image adapted from <a href="https://en.wikipedia.org/wiki/Matrix_multiplication_algorithm">Wikipedia</a></div>

This [wikipedia article](https://en.wikipedia.org/wiki/Computational_complexity_of_mathematical_operations) summarizes nicely the computational complexity of various algorithms for common mathematical operations.

> => libraries help using the most efficient algorithms for common tasks

**Complexity classes**

P:$\,\ \ \ \ \ \ \ \ \ \ \ \ \ \,$ solvable in polynomial time ("easy")<br>
NP: $\,\ \ \ \ \ \ \ \ \ \ \ $ solution is verifiable in polynomial time (e.g. integer factorization as decision problem)<br>
NP complete: the hardest problems in NP;<br> 
$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ $ any other NP-problem can be reduced to them in
polynomial time (e.g. traveling salesman)<br>
NP hard: $\ \ \ \ \ $ at least as hard as NP but not necessarily in NP (e.g. halting problem)<br><br>
many other classes exist, e.g. EXPTIME

<br>

![complexity classes](images/PNP.png)

<br>
<div style="text-align: center"> (simplified version; assuming deterministic computers and P ≠ NP)</div>


**P vs. NP**:
> If the solution to a problem is easy to check for correctness, must the problem be easy to solve?

*If P=NP, then the world would be a profoundly different place than we usually assume it to be. There would be no special value in "creative leaps," no fundamental gap between solving a problem and recognizing the solution once it's found. Everyone who could appreciate a symphony would be Mozart; everyone who could follow a step-by-step argument would be Gauss; everyone who could recognize a good investment strategy would be Warren Buffett.*

Scott Aaronson in his [Blog](https://www.scottaaronson.com/blog/?p=122)

### 1.4.3 Numerical roundoff

On a computer, numbers are not "exact":

In [ ]:
print('%.17f' % 0.1)

=> nearest representable binary fraction

"Regular" decimal fraction:

$$\frac{1}{8} = 0.125 = \frac{1}{10} + \frac{2}{100} + \frac{5}{1000}$$

Corresponding binary fraction:

$$\frac{1}{8} = 0.001 = \frac{0}{2} + \frac{0}{4} + \frac{1}{8}$$

- this is not a bug!
- most decimal fractions cannot be represented exactly as binary fractions
- this happens in all languages that support your hardware's floating point arithmetic
- many languages do not display the difference by default

$1/3 \ \ = 0.3333333333333\ldots\ \Rightarrow$ infinite decimal fraction

$1/10 = 0.0001100110011\ldots\ \Rightarrow$ infinite binary fraction

In [1]:
print('%.25f' % 0.5)
print('%.25f' % 0.51)

0.5000000000000000000000000
0.5100000000000000088817842


**Integers**: unlimited precision

Binary representation:

5 = 00000000 00000000 00000000 00000101

- stored as 2-complement from $-2^{n-1}$ to $2^{n-1}-1$ (with $n=32$ or higher)
- highest bit encodes the sign

In [3]:
import sys
sys.maxsize

9223372036854775807

(on my computer default 64bit representation)

In Python 3, larger integers are automatically converted to long ints, which are limited only by total available memory:

In [6]:
2**2000

114813069527425452423283320117768198402231770208869520047764273682576626139237031385665948631650626991844596463898746277344711896086305533142593135616665318539129989145312280000688779148240044871428926990063486244781615463646388363947317026040466353970904996558162398808944629605623311649536164221970332681344168908984458505602379484807914058900934776500429002716706625830522008132236281291761267883317206598995396418127021779858404042159853183251540889433902091920554957783589672039160081957216630582755380425583726015528348786419432054508915275783882625175435528800822842770817965453762184851149029376

In [5]:
float(2**2000)

OverflowError: int too large to convert to float

**Floating points**: correspond to double precision in C

![double precision floating point format](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/IEEE_754_Double_Floating_Point_Format.svg/1000px-IEEE_754_Double_Floating_Point_Format.svg.png)

_Image source:_ [wikipedia](https://en.wikipedia.org/wiki/Double-precision_floating-point_format)

Corresponds to
$$
(-1)^{\text{sign}}(1.b_{51}b_{50}...b_{0})_{2}\times 2^{e-2^{10}+1}
$$
or equivalently
$$
(-1)^{\text{sign}}\left(1+\sum _{i=1}^{52}b_{52-i}2^{-i}\right)\times 2^{e-2^{10}+1}
$$

- range: approximately $10^{-308}$ to $10^{308}$
- machine precision (machine epsilon $\varepsilon_m$): $2^{-52}\approx 2.2\cdot10^{-16}$
- $\varepsilon_m/2$ is the smallest relative gap between representable numbers
- $\varepsilon_m/2$ is **not** the smallest floating-point number that can be represented on a machine. That number depends on how many bits there are in the **exponent**, while $\varepsilon_m$ depends on how many bits there are in the **mantissa** (the fraction part).

In [7]:
import sys
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

In [8]:
smallNumber = 1.0+2.220446049250313e-16/2.
print('%.16f' % smallNumber)

1.0000000000000000


In [9]:
smallNumber = 1.0+2.220446049250314e-16/2.
print('%.16f' % smallNumber)

1.0000000000000002


Note that $\varepsilon_m$ applies to **significant digits**:

In [10]:
smallNumber = 1.0+2.220446049250314e-16/2.
print('%.16f' % smallNumber)
smallNumber = 10.0+2.220446049250314e-16/2.
print('%.16f' % smallNumber)

1.0000000000000002
10.0000000000000000


In [11]:
smallNumber = 10.0+22.20446049250314e-16/2.
print('%.16f' % smallNumber)

10.0000000000000018


In [12]:
import math
print(format(math.pi, '.10g'))   # give 10 significant digits
print(format(math.pi, '.10f'))   # give 10 digits after the point

3.141592654
3.1415926536


Infinity vs. overflow:

In [13]:
x = 10.0**200; y = x*x;
y

inf

In [14]:
y/y

nan

In [15]:
float('inf'), float('nan')

(inf, nan)

In [16]:
float( (10**200)*(10**200) )

OverflowError: int too large to convert to float

More details in the [IEEE 754](https://en.wikipedia.org/wiki/IEEE_754) Standard for Floating-Point Arithmetic

Floating point comparison:

In [17]:
(0.1 + 0.1 + 0.1) == 0.3

False

In [18]:
0.1 + 0.1 + 0.1

0.30000000000000004

Instead use:

In [19]:
def isclose(a, b, rel_tol=1e-09, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

print(isclose(2, 2.019, 0.01, 0), isclose(2, 2.019, 0.001,0))
print(isclose(2, 2.019, 0, 0.02), isclose(2, 2.019, 0, 0.002))

True False
True False


or in Python 3: ```math.isclose``` and ```cmath.isclose```, explained [here](https://docs.python.org/3/whatsnew/3.5.html#pep-485-a-function-for-testing-approximate-equality)

### 1.4.4 Errors and accuracy


In [20]:
1/49.0*49

0.9999999999999999

**Representation error**

- most decimal fractions cannot be represented exactly as binary fractions

**Roundoff error**
- roundoff errors are a characteristic of computer hardware
- pretty much any arithmetic operation among floating numbers should be thought of as introducing an additional fractional error of at least $\varepsilon_m$
- roundoff errors accumulate with increasing amounts of calculation
- best case: total roundoff on the order of $\sqrt{N}\varepsilon_m$ (cancellations, random walk)
- typical case: roundoff errors accumulate preferentially in one direction => total of order $N\varepsilon_m$
    - regularities of the calculation
    - peculiarities of your computer
- vast increase of roundoff error of single operations possible => see section on Numerical Stability

**Truncation error** (also called approximation error, or algorithmic error)

- is characteristic of the algorithm used;
- is independent of the hardware on which the program is executed;
- would exist even on a "perfect" infinitely accurate computer;
- can be entirely controlled by programmer.

Many numerical algorithms compute "discrete" approximations to some desired "continuous" quantity.

=> adjustable parameters (e.g. number of points, number of expansion terms)

=> true answer is obtained only when these parameters go to infinity

=> in practice use a finite, but sufficiently large choice

*$[\ldots]$ it is only a slight exaggeration to say that clever minimization of truncation error is practically the entire content of the field of numerical analysis!*
<div style="text-align: right">"Numerical Recipes in C" by W. H. Press <i>et al.</i> </div>

### 1.4.5 Numerical stability

**Example**: naive computation of a Taylor series

$$ e^x = 1 +x +\frac{x^2}{2!}+\frac{x^3}{3!}+\ldots$$

In [21]:
import numpy as np

def exponentialTaylorSeries(x):
    tolerance = 1e-15
    previousSum = 0
    currentSum = 1
    currentTerm = 1
    n = 0
    
    while(np.abs(currentSum - previousSum) > tolerance):
        n = n + 1
        currentTerm = currentTerm * x/n
        previousSum = currentSum
        currentSum += currentTerm
        if(n%5 == 1): print('{0:2d} \t {1:.6e} \t {2:.6g}'.format(n, currentTerm, currentSum))
            
    print("\n Naive power series: ",currentSum,"\n Correct answer:     ", np.exp(x))

In [22]:
exponentialTaylorSeries(-20.)

 1 	 -2.000000e+01 	 -19
 6 	 8.888889e+04 	 67736.6
11 	 -5.130672e+06 	 -3.27105e+06
16 	 3.132278e+07 	 1.71827e+07
21 	 -4.104743e+07 	 -1.97702e+07
26 	 1.664029e+07 	 7.14545e+06
31 	 -2.611609e+06 	 -1.01191e+06
36 	 1.847331e+05 	 65217.9
41 	 -6.573564e+03 	 -2131.53
46 	 1.278822e+02 	 38.3436
51 	 -1.451726e+00 	 -0.404809
56 	 1.013470e-02 	 0.00264125
61 	 -4.542815e-05 	 -1.11073e-05
66 	 1.355519e-07 	 3.68653e-08
71 	 -2.776299e-10 	 5.56139e-09
76 	 4.007323e-13 	 5.62197e-09
81 	 -4.170776e-16 	 5.62188e-09

 Naive power series:  5.621884390193919e-09 
 Correct answer:      2.061153622438558e-09


- most of the time, truncation and roundoff errors do not strongly interact
- but sometimes, early stage roundoff errors become magnified and obscure the true answer => unstable method
- an unstable method would work on a "perfect" computer, but not in practice
- typical cause: subtraction of two almost equal numbers => "catastrophic cancellation"

In [ ]:
1.0e20 + 0.1 - 1.0e20

In [ ]:
1.0e20 - 1.0e20 + 0.1

Stable alternatives:
- pairwise summation: "divide and conquer"
- Kahan summation (even smaller error, but slower): keeps separate variable to accumulate small errors

=> equivalent **mathematically** but not **numerically**!

In [ ]:
import math

myList = [1.0e20,0.1,-1.0e20]

print(sum(myList))         # naive summation
print(math.fsum(myList))   # stable summation

### Bonus: Numerical disasters

Incorrect numerics can cause huge financial loss and even loss of life:

- Patriot Missile failure (Dharan, Saudi Arabia, February 25, 1991)
    - 28 dead, approx. 100 injured
    - reason: poor handling of roundoff errors
    - time was measured in 0.1 second intervals: error of 0.000000095 per measurement on 24bit system
    - after 100 hours of operation: 0.34 seconds
    - rocket at 1676 m/s travels over 500 meters in this time
- explosion of the Ariane 5 rocket just after lift-off on its maiden voyage (June 4, 1996)
    - unmanned, but $\$$500 million value + a decade of development costing $\$$7 billion
    - reason: integer overflow
    - 64bit float was converted to 16bit signed integer
    - since the number was larger than 32768 the conversion failed

Source: https://cs.fit.edu/~ryan/library/Some_disasters_attributable_to_Numerical_Analysis.pdf

Less serious:

- In December 2014 the Gangnam Style music video broke the YouTube view limit (32bit integer) having been viewed more than 2,147,483,647 times (currently about 3.2 billion views)
- Youtube has now [changed the maximum view limit](https://plus.google.com/+YouTube/posts/BUXfdWqu86Q) to 9,223,372,036,854,775,808 (64bit integer)
- Current view record: just under 5.5 billion